In [ ]:
from __future__ import annotations

%matplotlib inline
import warnings

warnings.filterwarnings('ignore')

# Trends with the SBUV NOAA Dataset
Here we calculate trends using the GOZCARDS dataset by regressing to the VMR monthly zonal means using seasonal terms in our predictors.

In [ ]:
import numpy as np
import xarray as xr

from LOTUS_regression.dataset.sbuv import SBUV_NOAA
from LOTUS_regression.predictors import load_data
from LOTUS_regression.predictors.seasonal import add_seasonal_components
from LOTUS_regression.regression import regress_all_bins

#data = SBUV_NOAA(r'/home/runner/work/lotus-regression/lotus-regression/test_data//SBUV_NOAA/')
data = SBUV_NOAA('/home/kimberlee/Documents/lotus-regression/test_data/SBUV_NOAA/')

In [ ]:
data = data.loc[dict(mean_latitude=slice(-60, 60))]

data = data.where(data.time < np.datetime64('2013'), drop=True)

print(data)

In [ ]:
predictors = load_data('pred_baseline_pwlt.csv')

print(predictors.columns)

In [ ]:
predictors = add_seasonal_components(predictors, {'constant': 4, 'linear_pre': 2, 'linear_post': 2, 'qboA': 2, 'qboB': 2})

print(predictors[:5])

In [ ]:
results = regress_all_bins(predictors, data['vmr'], tolerance=0.1, debug=True)

# Convert to ~ percent
results /= data['vmr'].mean(dim='time')
results *= 100

print(results)

In [ ]:
import LOTUS_regression.plotting.trends as trends

trends.pre_post_with_confidence(results, x='mean_latitude', y='pressure', ylim=(100, 0.5), log_y=True, figsize=(16, 6),
                                x_label=r'Latitude [$^\circ$]', y_label='Pressure [hPa]', pre_title='Pre 1997',
                                post_title='Post 1997')